In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
gene_x_sample = pd.read_csv(PATH["gene_x_sample_file_path"], sep="\t", index_col=0)

target_x_sample = pd.read_csv(PATH["target_x_sample_file_path"], sep="\t", index_col=0)

gmt = ccal.read_gmts(PATH["gene_set_file_paths"])

In [ ]:
for target_name, target_values in target_x_sample.iterrows():

    output_directory_path = "../output/mountain_plot/{}".format(target_name)

    ccal.establish_path(output_directory_path, "directory")

    score_moe_p_value_fdr = pd.read_csv(
        "{}/{}/all.tsv".format(
            PATH["differentially_expressed_gene_directory_path"], target_name
        ),
        sep="\t",
        index_col=0,
    )

    gene_score = score_moe_p_value_fdr["Score"]

    gene_score = (gene_score - gene_score.mean()) / gene_score.std()

    for gene_set_name in SETTING["gene_set_to_peek"]:

        score = ccal.single_sample_gsea(
            gene_score,
            gmt.loc[gene_set_name],
            statistic="auc",
            title="{}: {}".format(target_name, gene_set_name),
            gene_score_name="Gene Score",
            html_file_path="{}/{}.html".format(output_directory_path, gene_set_name),
        )